# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
# TODO: Import the necessary libs
import os
import json
import time
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

# LLM + HTTP
import openai
import requests

# Optional Tavily SDK fallback
try:
    from tavily import TavilyClient
    _HAS_TAVILY_SDK = True
except Exception:
    _HAS_TAVILY_SDK = False


In [ ]:
# TODO: Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY") or OPENAI_API_KEY
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-3.5-turbo")

if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY is required in .env")

openai.api_key = OPENAI_API_KEY

In [ ]:
# -------------------------
# Chroma client & collection
# -------------------------
DB_PATH = os.getenv("CHROMA_PERSIST_PATH", "chromadb")  # persistent folder
chroma_client = chromadb.PersistentClient(path=DB_PATH)

# Use OpenAI embedding function. Make sure embedding API key is available.
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=CHROMA_OPENAI_API_KEY,
    model_name="text-embedding-ada-002"
)

COLLECTION_NAME = "udaplay"
collection = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_fn
)

# -------------------------
# Simple Memory
# -------------------------
MEMORY_FILE = "memory.jsonl"


def append_memory(record: Dict[str, Any]) -> None:
    """Append a JSON line to memory."""
    with open(MEMORY_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")



### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
# TODO: Create retrieve_game tool
# -------------------------
# Tools
# -------------------------

def retrieve_game(query: str, k: int = 3) -> List[Dict[str, Any]]:
    """
    Semantic search over the 'udaplay' Chroma collection.
    Returns a list of hits with 'id', 'document', 'metadata', 'distance'.
    """
    if collection is None:
        raise RuntimeError("Chroma collection not available")

    # Chroma returns lists nested for batch queries (we query single)
    results = collection.query(
        query_texts=[query],
        n_results=k,
        include=["documents", "metadatas", "ids", "distances"]
    )

    hits = []
    if results and "ids" in results and len(results["ids"]) > 0:
        for i in range(len(results["ids"][0])):
            hit = {
                "id": results["ids"][0][i],
                "document": results["documents"][0][i] if results.get("documents") else None,
                "metadata": results["metadatas"][0][i] if results.get("metadatas") else None,
                "distance": results["distances"][0][i] if results.get("distances") else None,
            }
            hits.append(hit)
    return hits


def _call_openai_judge(prompt: str, model: str = OPENAI_MODEL, temperature: float = 0.0) -> str:
    """Small wrapper to call OpenAI chat completion and return assistant text."""
    messages = [
        {"role": "system", "content": "You are a helpful, precise evaluator."},
        {"role": "user", "content": prompt}
    ]
    resp = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=512
    )
    return resp.choices[0].message["content"].strip()


#### Evaluate Retrieval Tool

In [ ]:
# TODO: Create evaluate_retrieval tool
def evaluate_retrieval(question: str, retrieved_docs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Uses an LLM to evaluate if retrieved documents are sufficient to answer the question.
    Returns a dict: { useful: bool, description: str }
    The LLM is instructed to return JSON only.
    """
    docs_summary = "\n\n".join(
        [f"ID: {d['id']}\nDoc: {d['document']}\nMeta: {json.dumps(d['metadata'], ensure_ascii=False)}" for d in retrieved_docs]
    ) or "(no documents)"

    prompt = (
        "Your task is to evaluate whether the following retrieved documents are sufficient to answer the user question.\n\n"
        f"Question: {question}\n\n"
        "Retrieved documents:\n"
        f"{docs_summary}\n\n"
        "Return a JSON object with fields:\n"
        " - useful: true or false (are these documents enough to provide a correct, evidence-backed answer?)\n"
        " - description: a short explanation of your judgement and what is missing (if anything)\n\n"
        "IMPORTANT: Return only valid JSON (no extra commentary)."
    )

    assistant_text = _call_openai_judge(prompt)
    # Try to parse JSON robustly:
    try:
        parsed = json.loads(assistant_text)
    except Exception:
        # fallback: wrap text in description if not JSON
        parsed = {"useful": False, "description": assistant_text}

    # Normalize
    if "useful" not in parsed:
        parsed["useful"] = bool(parsed.get("useful", False))
    return parsed

#### Game Web Search Tool

In [ ]:
# TODO: Create game_web_search tool
def game_web_search(question: str, max_results: int = 5) -> Dict[str, Any]:
    """
    Uses Tavily to do a web search when the Vector DB is insufficient.
    Returns the JSON Tavily reply (answer + results).
    """
    if not TAVILY_API_KEY:
        raise RuntimeError("TAVILY_API_KEY not set in environment")

    body = {
        "query": question,
        "include_answer": True,
        "include_raw_content": False,
        "max_results": max_results,
        "search_depth": "basic"
    }

    # Prefer tavily SDK if installed
    if _HAS_TAVILY_SDK:
        try:
            client = TavilyClient(api_key=TAVILY_API_KEY)
            resp = client.search(question, max_results=max_results, include_answer=True)
            return resp
        except Exception as e:
            # fallback to REST
            print("Tavily SDK failed, falling back to REST:", e)

    # REST fallback
    url = "https://api.tavily.com/search"
    headers = {"Authorization": f"Bearer {TAVILY_API_KEY}", "Content-Type": "application/json"}
    r = requests.post(url, headers=headers, json=body, timeout=15)
    r.raise_for_status()
    return r.json()

def compose_answer_from_web(self, question: str, web_resp: Dict[str, Any]) -> Dict[str, Any]:
        """
        Compose answer using web search (Tavily) plus optionally retrieved docs.
        Expects Tavily-style response (contains 'answer' and 'results').
        """
        # If tavily included an answer, use it as starting point
        tavily_answer = web_resp.get("answer") if isinstance(web_resp, dict) else None
        results = web_resp.get("results", []) if isinstance(web_resp, dict) else []

        # Build prompt with top results
        results_text = "\n\n".join([f"{r.get('title','')}\n{r.get('url','')}\n{r.get('content','')[:500]}" for r in results[:5]])
        prompt = (
            "You are UdaPlay. Using the following web search answer + top results, produce a structured JSON:\n"
            " - answer: short answer with citations (format: [source-index])\n"
            " - sources: list of objects {idx: int, title: str, url: str}\n"
            " - confidence: low/medium/high\n\n"
            f"Original question: {question}\n\n"
            f"Tavily quick answer: {tavily_answer}\n\nTop results:\n{results_text}\n\n"
            "Return only valid JSON."
        )
        assistant_text = _call_openai_judge(prompt, temperature=0.0)
        try:
            parsed = json.loads(assistant_text)
        except Exception:
            # fallback basic structure
            parsed = {
                "answer": tavily_answer or "No web answer available.",
                "sources": [{"idx": i+1, "title": r.get("title"), "url": r.get("url")} for i, r in enumerate(results[:5])],
                "confidence": "medium" if tavily_answer else "low"
            }
        return parsed

### Agent

In [ ]:
# TODO: Create your Agent abstraction using StateMachine
# -------------------------
# Answer composer (Agent)
# -------------------------
@dataclass
class AgentResult:
    question: str
    answer: str
    sources: List[Dict[str, Any]]
    used_tools: List[str]
    evaluation: Dict[str, Any]


class UdaPlayAgent:
    def __init__(self):
        self.memory = []  # short session memory; persisted via append_memory
        self.tools = {
            "retrieve_game": retrieve_game,
            "evaluate_retrieval": evaluate_retrieval,
            "game_web_search": game_web_search
        }

    def compose_answer_from_docs(self, question: str, docs: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Ask the LLM to produce a structured answer using the retrieved docs as context.
        Returns a dict with keys 'answer' and 'sources' (ids + summary)
        """
        docs_context = "\n\n".join([f"ID: {d['id']}\nText: {d['document']}\nMeta: {json.dumps(d['metadata'], ensure_ascii=False)}"
                                     for d in docs])
        prompt = (
            "You are UdaPlay: an AI research agent for the video game industry.\n"
            "Use ONLY the documents below as your evidence to answer the user's question. "
            "Produce a JSON object with keys:\n"
            " - answer: concise answer to the question\n"
            " - sources: list of objects {id: <doc id>, excerpt: <one-sentence excerpt>}\n"
            " - confidence: low/medium/high\n\n"
            f"Question: {question}\n\n"
            f"Documents:\n{docs_context}\n\n"
            "Return ONLY valid JSON."
        )
        assistant_text = _call_openai_judge(prompt, temperature=0.0)
        try:
            parsed = json.loads(assistant_text)
        except Exception:
            parsed = {"answer": assistant_text, "sources": [{"id": d["id"], "excerpt": (d["document"] or "")[:200]} for d in docs], "confidence": "low"}
        return parsed

    def compose_answer_from_web(self, question: str, web_resp: Dict[str, Any]) -> Dict[str, Any]:
        """
        Compose answer using web search (Tavily) plus optionally retrieved docs.
        Expects Tavily-style response (contains 'answer' and 'results').
        """
        # If tavily included an answer, use it as starting point
        tavily_answer = web_resp.get("answer") if isinstance(web_resp, dict) else None
        results = web_resp.get("results", []) if isinstance(web_resp, dict) else []

        # Build prompt with top results
        results_text = "\n\n".join([f"{r.get('title','')}\n{r.get('url','')}\n{r.get('content','')[:500]}" for r in results[:5]])
        prompt = (
            "You are UdaPlay. Using the following web search answer + top results, produce a structured JSON:\n"
            " - answer: short answer with citations (format: [source-index])\n"
            " - sources: list of objects {idx: int, title: str, url: str}\n"
            " - confidence: low/medium/high\n\n"
            f"Original question: {question}\n\n"
            f"Tavily quick answer: {tavily_answer}\n\nTop results:\n{results_text}\n\n"
            "Return only valid JSON."
        )
        assistant_text = _call_openai_judge(prompt, temperature=0.0)
        try:
            parsed = json.loads(assistant_text)
        except Exception:
            # fallback basic structure
            parsed = {
                "answer": tavily_answer or "No web answer available.",
                "sources": [{"idx": i+1, "title": r.get("title"), "url": r.get("url")} for i, r in enumerate(results[:5])],
                "confidence": "medium" if tavily_answer else "low"
            }
        return parsed

    def run(self, question: str, k: int = 3) -> AgentResult:
        """
        Top-level orchestration:
         - retrieve -> evaluate -> (if needed web search) -> finalize answer -> store memory
        """
        used_tools = []
        retrieved = []
        try:
            retrieved = self.tools["retrieve_game"](question, k=k)
            used_tools.append("retrieve_game")
        except Exception as e:
            print("retrieve_game error:", e)
            retrieved = []

        evaluation = {"useful": False, "description": "no evaluation performed"}
        try:
            evaluation = self.tools["evaluate_retrieval"](question, retrieved)
            used_tools.append("evaluate_retrieval")
        except Exception as e:
            print("evaluate_retrieval error:", e)

        final_answer = {}
        sources = []

        if evaluation.get("useful"):
            composed = self.compose_answer_from_docs(question, retrieved)
            final_answer = composed.get("answer") if isinstance(composed, dict) else str(composed)
            sources = composed.get("sources", [])
            used_tools.append("compose_from_docs")
        else:
            # fallback to web search
            try:
                web_resp = self.tools["game_web_search"](question, max_results=5)
                used_tools.append("game_web_search")
                composed = self.compose_answer_from_web(question, web_resp)
                final_answer = composed.get("answer")
                sources = composed.get("sources", [])
            except Exception as e:
                print("game_web_search error:", e)
                # As last resort, ask LLM to answer from retrieved docs even if 'not useful'
                composed = self.compose_answer_from_docs(question, retrieved)
                final_answer = composed.get("answer")
                sources = composed.get("sources", [])

        result = AgentResult(
            question=question,
            answer=final_answer,
            sources=sources,
            used_tools=used_tools,
            evaluation=evaluation
        )

        # Persist a short memory record
        mem_rec = {
            "timestamp": time.time(),
            "question": question,
            "answer": final_answer,
            "used_tools": used_tools,
            "evaluation": evaluation
        }
        append_memory(mem_rec)
        return result

In [ ]:
# TODO: Invoke your agent
# -------------------------
# Demo run for sample questions
# -------------------------
if __name__ == "__main__":
    agent = UdaPlayAgent()

    sample_questions = [
        "When were Pokémon Gold and Silver released?",
        "Which one was the first 3D platformer Mario game?",
        "Was Mortal Kombat X released for PlayStation 5?"
    ]

    for q in sample_questions:
        print("\n" + "=" * 80)
        print("Question:", q)
        try:
            res = agent.run(q, k=3)
            print("Answer (structured):")
            print(json.dumps({
                "answer": res.answer,
                "sources": res.sources,
                "evaluation": res.evaluation,
                "used_tools": res.used_tools
            }, ensure_ascii=False, indent=2))
        except Exception as exc:
            print("Agent run failed:", exc)

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes